# 05/05 - Pair Programming EDA 2 - Guadalupe & Lara

In [36]:
import pandas as pd
import re
import numpy as np

pd.options.display.max_rows


60

# Hipótesis

- ¿Es Australia el sitio más peligroso y letal para estar relajada en la playa?
- ¿Hay diferencias entre los paises en los ataques a hombres y mujeres?
- ¿Cuáles son las edades que más sufren ataques?
- Independientemente de la edad, ¿sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

In [15]:
df = pd.read_csv("ataques_enproceso.csv", index_col = 0)

df.head()

,case_number,year,type,country,area,location,activity,name,sex_,age,time,species_,fatal_(y/n),injury,date
0,2018.06.25,2018.0,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,18h00,White shark,N,"No injury to occupant, outrigger canoe and pad...",25-Jun-2018
1,2018.06.18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,14h00 -15h00,NaN,N,Minor injury to left thigh,18-Jun-2018
2,2018.06.09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,07h45,NaN,N,Injury to left lower leg from surfboard skeg,09-Jun-2018
3,2018.06.08,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,NaN,2 m shark,N,Minor injury to lower leg,08-Jun-2018
4,2018.06.04,2018.0,Provoked,mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,NaN,"Tiger shark, 3m",N,Lacerations to leg & hand shark PROVOKED INCIDENT,04-Jun-2018


# Objetivos

Usad la librería sidetable.

In [3]:
import sidetable as sdt

- Explorad los valores nulos.

In [4]:
print("Estos son valores nulos de cada columna:", "\n", df.isna().sum())

Estos son valores nulos de cada columna: 
 case_number       1
year              4
type              8
country          54
area            458
location        543
activity        546
name            213
sex_            568
age            2824
time           3344
species_       2835
fatal_(y/n)     537
injury           30
date              2
dtype: int64


In [5]:
print("El porcentaje de nulos por columna es:")
porcentaje = (df.isna().sum() / df.shape[0] * 100).round(2)
porcentaje

El porcentaje de nulos por columna es:


case_number     0.02
year            0.06
type            0.13
country         0.86
area            7.28
location        8.64
activity        8.68
name            3.39
sex_            9.03
age            44.91
time           53.18
species_       45.09
fatal_(y/n)     8.54
injury          0.48
date            0.03
dtype: float64

- Explorad la columna de country y reportad los resultados solo para aquellas categorías que incluyan el 70% de los datos.

In [6]:
# Exploramos la columna con sidetable freq, para ver la frecuencia de cada valor para "country".
freq_country = df.stb.freq(["country"])
cat_70 = freq_country[freq_country["cumulative_percent"] < 71]
cat_70

,country,count,percent,cumulative_count,cumulative_percent
0,usa,2222,35.643247,2222,35.643247
1,australia,1335,21.414822,3557,57.058069
2,south africa,577,9.255695,4134,66.313763
3,papua new guinea,134,2.149503,4268,68.463266
4,new zealand,127,2.037215,4395,70.500481


👩🏽‍🦰👩🏽‍🦱💬
- Para mostrar **todas las columnas de df** para los **países que representan el 70 %** del total de los datos, filtramos el dataframe original con la condición obtenida en la celda anterior:

In [7]:
info_70 = df[df["country"].isin(cat_70["country"])]
info_70.head(2)

,case_number,year,type,country,area,location,activity,name,sex_,age,time,species_,fatal_(y/n),injury,date
0,2018.06.25,2018.0,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,18h00,White shark,N,"No injury to occupant, outrigger canoe and pad...",25-Jun-2018
1,2018.06.18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,14h00 -15h00,NaN,N,Minor injury to left thigh,18-Jun-2018


- Explorad la columna country en relación al sexo, filtrando de nuevo por aquellas categorías que incluyan el 70% de los datos.

In [8]:
# Exploramos la columna "country" respecto al "sex_" con sidetable freq, para ver la frecuencia de cada valor del sexo (F/M) en cada país.

freq_country_sex = df.stb.freq(["country", "sex_"])
freq_country_sex.head(8)

,country,sex_,count,percent,cumulative_count,cumulative_percent
0,usa,M,1787,31.494536,1787,31.494536
1,australia,M,1104,19.457173,2891,50.951710
2,south africa,M,487,8.583010,3378,59.534720
3,usa,F,336,5.921748,3714,65.456468
4,papua new guinea,M,108,1.903419,3822,67.359887
5,new zealand,M,100,1.762425,3922,69.122312
6,brazil,M,93,1.639055,4015,70.761368
7,bahamas,M,92,1.621431,4107,72.382799


In [9]:
# Reportamos los datos de aquellos países que nos ofrecen el 70 % del total de los datos (a través del porcentaje acumulado). Son los 6 primeros.
#  [* El valor de Brazil lo dejamos fuera porque muestra más del 70%, ¿cogeríamos una parte de los datos de Brasil? ¿Con una regla de tres?]

cat_70_sex = freq_country_sex[freq_country_sex["cumulative_percent"] <= 70]
cat_70_sex.round()

,country,sex_,count,percent,cumulative_count,cumulative_percent
0,usa,M,1787,31.0,1787,31.0
1,australia,M,1104,19.0,2891,51.0
2,south africa,M,487,9.0,3378,60.0
3,usa,F,336,6.0,3714,65.0
4,papua new guinea,M,108,2.0,3822,67.0
5,new zealand,M,100,2.0,3922,69.0


👩🏽‍🦰👩🏽‍🦱💬

- Mostramos **todas las columnas de df** de aquellos países que nos ofrecen el 70 % del total de los datos, desglosados por sexo. 



In [10]:
info_70_s = df[df["country"].isin(cat_70_sex["country"])]
info_70_s.head(2)

,case_number,year,type,country,area,location,activity,name,sex_,age,time,species_,fatal_(y/n),injury,date
0,2018.06.25,2018.0,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,18h00,White shark,N,"No injury to occupant, outrigger canoe and pad...",25-Jun-2018
1,2018.06.18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,14h00 -15h00,NaN,N,Minor injury to left thigh,18-Jun-2018


- Haced los mismo para la columna activity en relación a la columna fatal. En este caso filtrad y mostrad los datos que incluyan el 60% de los datos.

In [27]:
freq_act_fatal = df.stb.freq(["country", "fatal_(y/n)"])
freq_act_fatal.head(8)

,country,fatal_(y/n),count,percent,cumulative_count,cumulative_percent
0,usa,N,1832,32.112182,1832,32.112182
1,australia,N,913,16.003506,2745,48.115688
2,south africa,N,406,7.116564,3151,55.232252
3,australia,Y,283,4.960561,3434,60.192813
4,usa,Y,185,3.242770,3619,63.435583
5,south africa,Y,106,1.858019,3725,65.293602
6,bahamas,N,94,1.647677,3819,66.941280
7,new zealand,N,90,1.577564,3909,68.518843


In [29]:
act_fatal_60 = freq_act_fatal[(freq_act_fatal["cumulative_percent"]) < 61]
act_fatal_60.round()

,country,fatal_(y/n),count,percent,cumulative_count,cumulative_percent
0,usa,N,1832,32.0,1832,32.0
1,australia,N,913,16.0,2745,48.0
2,south africa,N,406,7.0,3151,55.0
3,australia,Y,283,5.0,3434,60.0


- Por último, realizad este análisis para la columna fecha (la que creamos en el último ejercicio de pair programming de Pandas) en relación a la columna fatal. Mostrad solo los resultados para las categorías que incluyan el 60% de los datos.

In [40]:
# Exploramos la columna "date" respecto a "fatal_(y/n)" con sidetable freq, para ver la frecuencia de cada valor de "fatal" y cada actividad.

freq_date_fatal = df.stb.freq(["date", "fatal_(y/n)"])
freq_date_fatal.head()


,date,fatal_(y/n),count,percent,cumulative_count,cumulative_percent
0,1957,N,6,0.104330,6,0.104330
1,1950,N,6,0.104330,12,0.208659
2,1942,N,6,0.104330,18,0.312989
3,No date,N,5,0.086941,23,0.399930
4,28-Jul-1995,N,5,0.086941,28,0.486872


#DRAMAAAAAAAAA
- ¡La columna no está limpia! 🚒 Nuestra sabiduría EDA nos indica que no podemos hacer un análisis correcto todavía (la limpieza de esta columna por meses se realiza en EDA_III, por lo que en ese pair analizaremos la distribución de ataques a lo largo del año), aunque sí mostrar el 60 % de los resultados según el estado de la columna ahora.


In [47]:
date_fatal_60 = freq_date_fatal[(freq_date_fatal["cumulative_percent"]) <= 60]
date_fatal_60.round()


,date,fatal_(y/n),count,percent,cumulative_count,cumulative_percent
0,1957,N,6,0.0,6,0.0
1,1950,N,6,0.0,12,0.0
2,1942,N,6,0.0,18,0.0
3,No date,N,5,0.0,23,0.0
4,28-Jul-1995,N,5,0.0,28,0.0
...,...,...,...,...,...,...
2859,19-Jul-1847,Y,1,0.0,3446,60.0
2860,19-Jan-2015,N,1,0.0,3447,60.0
2861,19-Jan-2012,N,1,0.0,3448,60.0
2862,19-Jan-2005,N,1,0.0,3449,60.0


📌 Nota Igual que siempre, no nos vale solo con picar el código, queremos que nos incluyais las conclusiones que extraeis.

# Conclusiones
Si bien es cierto que la lección de hoy ha sido muy teórica, en esta parte los ejercicios no serán tanto de programar sino de pensar 🤔. ¿Qué tendréis que hacer?

En relación a los valores nulos:
Identificar las columnas que tienen nulos (lo hicimos en el ejercicio de pair programming EDA I y en el de hoy con el sidetable).

Debatid entre vosotras qué haríais con los valores nulos, ¿los quitaríais o los reemplazaríais con un valor que nosotras decidiremos?. Cread una celda de markdown en vuestro jupyter y escribid vuestras conclusiones.

👩🏽‍🦰👩🏽‍🦱💬 
**AQUÍ SE DESARROLLARÁ NUESTRO DEBATE**
🚒 🚒 🚒 Magia is welcome 🚒 🚒 🚒 

En relación a los outliers, ya hemos ido conociendo un poco nuestro DataFrame y de momento solo tenemos una columna de tipo numérico, por lo tanto, poco podemos hacer. Un poco más adelante, cuando hayamos hecho una serie de cambios sobre nuestras columnas podremos hacer una evaluación más profunda de estos.